In [ ]:
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler  # z-нормировка
# from sklearn.preprocessing import MinMaxScaler, RobustScaler, MaxAbsScaler # другие способы нормировки

from statsmodels.tsa.api import ARIMA

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

Будем использовать модель [MLP](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) из пакета [scikit-learn](https://scikit-learn.org/stable/index.html)

In [ ]:
def make_datasets(data, n_lags=2, steps=1, gap=0):
	y = np.full((len(data)-n_lags-steps-gap, steps), 0.0)
	X = np.full((len(data)-n_lags-steps-gap, n_lags), 0.0)

	for i in range(n_lags):
		X[:,i] = data[i:len(data)-n_lags-steps-gap+i]

	for i in range(steps):
		y[:,i] = data[n_lags+gap+i:len(data)-steps+i]

	return X, y

In [ ]:
rate = web.DataReader(name='WGS10YR', data_source='fred', start='2000-01-01')
rate.plot()

In [ ]:
series = rate.iloc[:,0].values

lags = 50
fh = 10

X, y = make_datasets(series, n_lags=lags, steps=fh)

In [ ]:
forecaster = MLPRegressor(hidden_layer_sizes=(75,), activation='relu', random_state=1, max_iter=2000)

forecaster.fit(X,y)

y_pred_mlp = forecaster.predict(np.reshape(series[-lags:], (1, lags))).T

In [ ]:
last_obs = 50
plt.plot(np.arange(len(series)-last_obs, len(series)), series[-last_obs:])
plt.plot(np.arange(len(series), len(series)+fh), y_pred_mlp)
plt.show()

# MLP vs ARIMA

In [ ]:
# спецификация модели
mod = ARIMA(series, order=(2,1,2), trend='n', missing='drop')
# подгонка модели на данных
res = mod.fit()

y_pred_arima = res.forecast(steps=fh)

In [ ]:
last_obs = 50
plt.plot(np.arange(len(series)-last_obs, len(series)), series[-last_obs:])
plt.plot(np.arange(len(series), len(series)+fh), y_pred_mlp)
plt.plot(np.arange(len(series), len(series)+fh), y_pred_arima)
plt.show()

# Кросс-валидация

In [ ]:
test_size = 10

lags = 50

series_train = series[:-test_size]

X_train, y_train = make_datasets(series_train, n_lags=lags, steps=test_size)

y_test = series[-test_size:]
X_test = np.reshape(series[-test_size-lags:-test_size], (1, lags))

In [ ]:
forecaster = MLPRegressor(hidden_layer_sizes=(75,10), activation='relu', random_state=1, max_iter=2000)

forecaster.fit(X_train,y_train)

y_pred_mlp = forecaster.predict(X_test).T

# спецификация модели
mod = ARIMA(series_train, order=(2,1,2), trend='n', missing='drop')
# подгонка модели на данных
res = mod.fit()

y_pred_arima = res.forecast(steps=test_size)

In [ ]:
last_obs = 100

plt.plot(np.arange(len(series)-last_obs, len(series)), series[-last_obs:])
plt.plot(np.arange(len(series)-test_size, len(series)), y_pred_mlp)
plt.plot(np.arange(len(series)-test_size, len(series)), y_pred_arima)
plt.show()

In [ ]:
mean_squared_error(y_test, y_pred_mlp), mean_squared_error(y_test, y_pred_arima)

In [ ]:
mean_absolute_error(y_test, y_pred_mlp), mean_absolute_error(y_test, y_pred_arima)

In [ ]:
mean_absolute_percentage_error(y_test, y_pred_mlp), mean_absolute_percentage_error(y_test, y_pred_arima)